<a href="https://colab.research.google.com/github/DGkurnia/discuss1/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#libary import
import pandas as pd  # Untuk load & mengolah tabel/CSV/Excel/Gsheet
import numpy as np   # Numpy in Array
import matplotlib.pyplot as plt
import seaborn as sns
import shutil   #managing ZIP files
from datetime import datetime # for managing time-related date
#importing tables
dfcustomer = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/customers.csv")
dforder = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/orders.csv")
dfproduct = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/products.csv")
dfsale = pd.read_csv("https://raw.githubusercontent.com/dicodingacademy/dicoding_dataset/main/DicodingCollection/sales.csv")
#inspecting each df
dfsale.head(10)


,sales_id,order_id,product_id,price_per_unit,quantity,total_price
0,0,1,218,106,2,212.0
1,1,1,481,118,1,118.0
2,2,1,2,96,3,288.0
3,3,1,1002,106,2,212.0
4,4,1,691,113,3,339.0
5,5,1,981,106,3,318.0
6,6,2,915,96,1,96.0
7,7,2,686,113,1,113.0
8,8,2,1091,115,3,345.0
9,9,2,1196,105,1,NaN


1. Preparing dataset

2. Fixing Missing values

In [ ]:
#inspecting missing data
dfcustomer.info()
dforder.info()
tnacol = dfcustomer.isna().sum()
print(tnacol)
print("Jumlah duplikasi: ", dfcustomer.duplicated().sum())
print("Jumlah duplikasi: ",dforder.duplicated().sum())
dforder.describe()
dfproduct.info()
print("Jumlah duplikasi: ", dfproduct.duplicated().sum())
dfproduct.describe()
dfsale.isna().sum()
"""
oke, sampai tahap ini, kita telah berhasil mengidentifikasi berbagai masalah pada data yang telah kita kumpulkan. Berikut merupakan rangkumannya.


Tipe data

Missing value (Terdapat 18 missing values pada kolom gender di
customer_df.)

Duplicate data (Terdapat 6 data yang duplikat.)

Inaccurate value Terdapat inaccurate value pada kolom age.

orders_df

Terdapat kesalahan tipe data untuk kolom order_date & delivery_date di product_df

di sales_df

Terdapat 19 missing value pada kolom total_price.
"""

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   customer_id    1007 non-null   int64 
 1   customer_name  1007 non-null   object
 2   gender         989 non-null    object
 3   age            1007 non-null   int64 
 4   home_address   1007 non-null   object
 5   zip_code       1007 non-null   int64 
 6   city           1007 non-null   object
 7   state          1007 non-null   object
 8   country        1007 non-null   object
dtypes: int64(3), object(6)
memory usage: 70.9+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   order_id       1000 non-null   int64 
 1   customer_id    1000 non-null   int64 
 2   payment        1000 non-null   int64 
 3   order_date     1000 non-null   object
 4   de

'\noke, sampai tahap ini, kita telah berhasil mengidentifikasi berbagai masalah pada data yang telah kita kumpulkan. Berikut merupakan rangkumannya.\n\n\nTipe data\n\nMissing value (Terdapat 18 missing values pada kolom gender di\ncustomer_df.)\n\nDuplicate data (Terdapat 6 data yang duplikat.)\n\nInaccurate value Terdapat inaccurate value pada kolom age.\n\norders_df\n\nTerdapat kesalahan tipe data untuk kolom order_date & delivery_date di product_df\n\ndi sales_df\n\nTerdapat 19 missing value pada kolom total_price.\n'

3. Handling mistakes in data

In [ ]:
dfcustomer.drop_duplicates(inplace=True)
#print("Jumlah duplikasi: ", dfcustomer.duplicated().sum())
#dfcustomer segment
dfcustomer.gender.value_counts()
#dfcustomer[dfcustomer.gender.isna()]
dfcustomer.fillna(value="Prefer not to say", inplace=True)
#managing incorrect age
dfcustomer.age.replace(dfcustomer.age.max(), 70, inplace=True)
dfcustomer.age.replace(dfcustomer.age.max(), 50, inplace=True)
#dfcustomer.describe()
#order segment
datetime_columns = ["order_date", "delivery_date"]

for column in datetime_columns:
  dforder[column] = pd.to_datetime(dforder[column])
#dforder.info()
#product segment
dfproduct.drop_duplicates(inplace=True)
#print("Jumlah duplikasi: ", dfproduct.duplicated().sum())
#sale segment
dfsale[dfsale.total_price.isna()]
dfsale["total_price"] = dfsale["price_per_unit"] * dfsale["quantity"]
dfsale.isna().sum()

,0
sales_id,0
order_id,0
product_id,0
price_per_unit,0
quantity,0
total_price,0


4. Describing code

In [ ]:
#describibng data
#dfcustomer.describe(include="all")
dfcustomer.groupby(by="gender").agg({
    "customer_id": "nunique",
    "age": ["max", "min", "mean", "std"]
})
#segmenting
dfcustomer.groupby(by="city").customer_id.nunique().sort_values(ascending=False)
dfcustomer.groupby(by="state").customer_id.nunique().sort_values(ascending=False)

,customer_id
state,
South Australia,139
Queensland,134
New South Wales,132
Northern Territory,125
Western Australia,124
Australian Capital Territory,121
Victoria,121
Tasmania,104


describing data - part 2

In [ ]:
#intro
delivery_time = dforder["delivery_date"] - dforder["order_date"]
delivery_time = delivery_time.apply(lambda x: x.total_seconds())
dforder["delivery_time"] = round(delivery_time/86400)
#inspecting
#print(dforder["delivery_time"])
#customer inspection
customer_id_in_orders_df =  dforder.customer_id.tolist()
dfcustomer["status"] = dfcustomer["customer_id"].apply(lambda x: "Active" if x in customer_id_in_orders_df else "Non Active")
dfcustomer.groupby(by="status").customer_id.count()
#merging dataset
orders_customers_df = pd.merge(
    left= dforder,
    right= dfcustomer,
    how="left",
    left_on="customer_id",
    right_on="customer_id"
)
orders_customers_df.head()

,order_id,customer_id,payment,order_date,delivery_date,delivery_time,customer_name,gender,age,home_address,zip_code,city,state,country,status
0,1,64,30811,2021-08-30,2021-09-24,25.0,fulan 64,Prefer not to say,75,4927 Alice MeadowApt. 960,7787,Sanfordborough,South Australia,Australia,Active
1,2,473,50490,2021-02-03,2021-02-13,10.0,fulan 473,Male,61,531 Schmitt BoulevardApt. 010,1744,Annaton,South Australia,Australia,Active
2,3,774,46763,2021-10-08,2021-11-03,26.0,fulan 774,Prefer not to say,34,2096 Wilson MewsApt. 714,8590,West Jonathanshire,Tasmania,Australia,Active
3,4,433,39782,2021-05-06,2021-05-19,13.0,fulan 433,Prefer not to say,46,5777 Mayer PassApt. 881,9728,West Michaelport,Tasmania,Australia,Active
4,5,441,14719,2021-03-23,2021-03-24,1.0,fulan 441,Prefer not to say,53,33 Richards JunctionApt. 478,7650,South Rileyview,Western Australia,Australia,Active


Pivot segmentation

In [ ]:
#city-based pivot
orders_customers_df.groupby(by="city").order_id.nunique().sort_values(ascending=False).reset_index().head(10)
#state based pivot
orders_customers_df.groupby(by="state").order_id.nunique().sort_values(ascending=False)
#gender based
orders_customers_df.groupby(by="gender").order_id.nunique().sort_values(ascending=False)
#age based group
orders_customers_df["age_group"] = orders_customers_df.age.apply(lambda x: "Youth" if x <= 24 else ("Seniors" if x > 64 else "Adults"))
orders_customers_df.groupby(by="age_group").order_id.nunique().sort_values(ascending=False)

,order_id
age_group,
Adults,681
Seniors,226
Youth,93


product and sales

In [ ]:
#sales prediction (phase 1)
dfproduct.sort_values(by="price", ascending=False)
#pivot in sales and product
dfproduct.groupby(by="product_type").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price":  ["min", "max"]
})

#determining
dfproduct.groupby(by="product_name").agg({
    "product_id": "nunique",
    "quantity": "sum",
    "price": ["min", "max"]
})
#merging the dataset
sales_product_df = pd.merge(
    left= dfsale,
    right= dfproduct,
    how="left",
    left_on="product_id",
    right_on="product_id"
)
sales_product_df.head(10)

,sales_id,order_id,product_id,price_per_unit,quantity_x,total_price,product_type,product_name,size,colour,price,quantity_y,description
0,0,1,218,106,2,212,Shirt,Chambray,L,orange,105,44,"A orange coloured, L sized, Chambray Shirt"
1,1,1,481,118,1,118,Jacket,Puffer,S,indigo,110,62,"A indigo coloured, S sized, Puffer Jacket"
2,2,1,2,96,3,288,Shirt,Oxford Cloth,M,red,114,54,"A red coloured, M sized, Oxford Cloth Shirt"
3,3,1,1002,106,2,212,Trousers,Wool,M,blue,111,52,"A blue coloured, M sized, Wool Trousers"
4,4,1,691,113,3,339,Jacket,Parka,S,indigo,119,53,"A indigo coloured, S sized, Parka Jacket"
5,5,1,981,106,3,318,Trousers,Wool,S,red,111,65,"A red coloured, S sized, Wool Trousers"
6,6,2,915,96,1,96,Trousers,Drawstring,XS,orange,104,41,"A orange coloured, XS sized, Drawstring Trousers"
7,7,2,686,113,1,113,Jacket,Parka,S,blue,119,72,"A blue coloured, S sized, Parka Jacket"
8,8,2,1091,115,3,345,Trousers,Cropped,S,orange,99,78,"A orange coloured, S sized, Cropped Trousers"
9,9,2,1196,105,1,105,Trousers,Pleated,S,orange,90,72,"A orange coloured, S sized, Pleated Trousers"


Pivot table from sales and products

In [ ]:
#preparing pivot from product typer
sales_product_df.groupby(by="product_type").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
})
#preparing pivot from name
sales_product_df.groupby(by="product_name").agg({
    "sales_id": "nunique",
    "quantity_x": "sum",
    "total_price": "sum"
}).sort_values(by="total_price", ascending=False)

,sales_id,quantity_x,total_price
product_name,,,
Denim,273,527,52399
Casual Slim Fit,154,306,36414
Trench Coat,146,299,35581
Shearling,150,302,35334
Puffer,140,298,35164
Flannel,141,281,33158
Cropped,135,284,32660
Pleated,147,308,32340
Joggers,164,334,31062


Merging all dataset

In [ ]:
#first step
all_df = pd.merge(
    left = sales_product_df,
    right = orders_customers_df,
    how="left",
    left_on="order_id",
    right_on="order_id"
)
#all_df.head(10)
all_df.groupby(by=["state", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})
all_df.groupby(by=["gender", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

all_df.groupby(by=["age_group", "product_type"]).agg({
    "quantity_x": "sum",
    "total_price": "sum"
})

quantity_x  total_price
age_group product_type                         
Adults    Jacket              2292       245055
          Shirt               2225       227781
          Trousers            2272       231271
Seniors   Jacket               777        82959
          Shirt                733        74977
          Trousers             766        77471
Youth     Jacket               274        29012
          Shirt                301        30842
          Trousers             322        32432